In [1]:
import qiskit as qk
import qutip as qt
import numpy as np
import numpy.linalg as la
import pandas as pd

In [2]:
# define hamiltonian
def TFIM_hamiltonian(n, J, gamma):
    # uses ring shape
    id = qt.qeye(2)
    z = qt.sigmaz()
    x = qt.sigmax()
    sxi = []
    szi = []
    for i in range(n):
        sxi.append(qt.tensor([id] * i + [x] + [id] * (n - i - 1)))
        szi.append(qt.tensor([id] * i + [z] + [id] * (n - i - 1)))
    return J * sum(szi[i] * szi[i + 1] for i in range(n - 1)) - szi[n - 1] * szi[0] - gamma * sum(sxi[i] for i in range(n))

In [3]:
test = TFIM_hamiltonian(3, 1, 1)
test.full()
la.eig(test.full())[0]


array([ 3.46410162e+00+0.j,  2.00000000e+00+0.j,  2.98119290e-16+0.j,
       -4.00000000e+00+0.j, -3.46410162e+00+0.j,  2.57072065e-16+0.j,
        2.00000000e+00+0.j,  1.23804543e-16+0.j])

In [4]:
# classically
def ground_state(h):
    results = la.eig(h)
    eigenvalues = results[0]
    Phi = results[1]
    min_index = 0
    for i in range(len(eigenvalues)):
        if np.real(eigenvalues[i]) < np.real(eigenvalues[min_index]):
            min_index = i
    return np.real(eigenvalues[min_index]), Phi[:, min_index]

In [5]:
ground_state(test.full())

(np.float64(-3.9999999999999947),
 array([0.20412415-0.j, 0.20412415-0.j, 0.61237244-0.j, 0.20412415-0.j,
        0.20412415-0.j, 0.61237244+0.j, 0.20412415-0.j, 0.20412415-0.j]))

In [7]:
# quantum method, using either QAOA or adiabatic
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import SPSA